In [1]:
#It has already being known that Surface Temperatures (ST) can distinguish habitable planets from non
#habitable ones with a large degree of precision. So it is necessary to verify whether our classifier works properly even without
#this feature
import pandas as pd
#reading the cleaned dataset
df=pd.read_csv("output.csv")

In [2]:
#deleting columns which contains temperature
del df['P. Teq Min (K)'],df['P. Teq Mean (K)'],df['P. Teq Max (K)'],df['P. Ts Min (K)'],df['P. Ts Mean (K)'],df['P. Ts Max (K)']


In [3]:
#applying upsampling
from sklearn.utils import resample
df_majority=df[df['target']==0]
df_minority_1=df[df['target']==1]
df_minority_2=df[df['target']==2]
# Upsample minority class
df_minority_upsampled_1 = resample(df_minority_1, 
                                 replace=True,  
                                 n_samples=1000,
                                 random_state=123) 
df_minority_upsampled_2 = resample(df_minority_2, 
                                 replace=True,  
                                 n_samples=1000,
                                 random_state=123) 
df_upsampled = pd.concat([df_majority, df_minority_upsampled_1,df_minority_upsampled_2])

In [4]:
print(df_upsampled.shape[0])
df_upsampled.to_csv("upsampling1.csv")

3747


In [5]:
#CLASSIFICATION USING KNN(k=3)
#loading the dataset and splitting the dataset into test and train data

import csv
import random
def loadDataset(filename,trainset=[] , testset=[]):
    lines = csv. reader(open(filename ,'r'))
    dataset = list(lines)
    dataset.remove(dataset[0]) #first element is column names so removing that perticular element from dataset
    k=len(dataset[0])
    for x in range(len(dataset)):
        for y in range(k):
                dataset[x][y] = float(dataset[x][y]) #converting all values from string into float
    l=[]
    for i in range(0,1256):  #splitting the datset into test and train where the splitting ratio is 0.7
        l.append(random.randint(0,len(dataset)-1))
    for j in range(0,len(dataset)-1):
        if j in l:
            trainset.append(dataset[j])
        else:
            testset.append(dataset[j])





trainset=[]
testset=[]
loadDataset('upsampling1.csv', trainset, testset)

In [6]:
#calculating euclidian distance
import math
def find_distance(testpoint,trainpoint):
    dist=0
    no_attributes=len(testpoint)
    for i in range(no_attributes-1):
        dist=dist+pow((testpoint[i]-trainpoint[i]),2)
    value=math.sqrt(dist)
    return value

In [7]:
#finding nearest k neighbors
import operator 
def findNeighbors(trainset,testpoint,k):
    distances=[]
    for x in range(0,len(trainset)):
        dist=find_distance(testpoint,trainset[x])
        distances.append((trainset[x],dist))
    
    distances.sort(key=operator.itemgetter(1))
    nearest_points=[]
    for j in range(k):
        nearest_points.append(distances[j][0])
    return nearest_points

In [8]:
#finding the majority class among k neighbors
def predict(neighbors):
    target_values={}
    k=len(neighbors[0])
    for ele in neighbors:
        target=ele[k-1]
        if target in target_values:
            target_values[target]+=1
        else:
            target_values[target]=1
    sortedvalues = sorted(target_values.items(), key=operator.itemgetter(1), reverse=True)
    return sortedvalues[0][0]

In [9]:
#calculating the predicted value
predicted_value=[]
k=3
for x in range(len(testset)):
    neighbors = findNeighbors(trainset, testset[x], k)
    result = predict(neighbors)
    predicted_value.append(result)

In [10]:
# calculating the accuracy
def find_accuracy(testset,predicted_value):
    count=0
    k=len(testset[0])
    for i in range(len(testset)):
        if testset[i][k-1]==predicted_value[i]:
            count=count+1
    final_accuracy=(count/float(len(testset)))*100
    return final_accuracy

In [11]:
# calculating the accuracy
accuracy=find_accuracy(testset,predicted_value)
print("accuracy= ",accuracy)

accuracy=  96.95732838589981
